* Chain in which we execute with retrival set up 
* Agent in which we give an LLLM discretion over wheather and how to execute a retrival step (or multiple step )

In [5]:
import os
from  dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=groq_api_key,model_name="Llama3-8b-8192")

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [13]:
## load , chunk and index the contents of the blog to create a retriever
import bs4
loader = WebBaseLoader(
    web_path=("https://groq.com/careers/",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(class_=("elementor-widget-container","job-posts","job-posts--table")
        )
    ),
)
docs = loader.load()
docs

[Document(page_content='\n\n\n\n\n\n\n\nNow in Preview: Groq’s First Compound AI System. \n\n\n\n\n\n\nTry Now &\n\n\n\n\n\n\n\n\n\n\nLearn More. \n\n\n\n\n\n\n\n\n\n\n\n\n\nLlama 4 is live on GroqCloud™. \n\n\n\n\n\n\nLearn more &\n\n\n\n\n\n\n\n\n\n\ntry now.\n\n\n\n\n\n\n\n\n\n\n\n\n\nGroq Launches First TTS Model, PlayAI Dialog, Making Voice AI Sound More Human.  \n\n\n\n\n\n\nTry Now &\n\n\n\n\n\n\n\n\n\n\nLearn More\n\n\n\n\n\n\n\n\n\n\n\n\n\nGroqCloud™ Self-serve Developer Tier now available! \n\n\n\n\n\n\nLearn more.\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n\n\n\n \n\n \n\n\n\n\n\n \n\t\t\t\t\t\t\tProducts\t\t\t\t\t\t\n \n\n Close Products\n\n\n Open Products\n\n\n\n\n\n\n\n\n\n\nPricing\n\n\n\n\nProducts Overview\n\n\n\n\nEnterprise Access\n\n\n\n\nGroqCloud™ Platform\n\n\n\n\nGroqRack™ Cluster\n\n\n\n\n\n\n\n\n\n\n \n\t\t\t\t\t\t\tDevelopers\t\t\t\t\t\t\n \n\n Close Developers\n\n\n Open Developers\n\n\n\n\n\n\n\n\n\n\n\nFree API Key \xa0\n\n\n\n\nStart Building \xa0\n\n\n\n\nDiscord

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f"Number of splits: {len(splits)}")


Number of splits: 6


In [15]:
splits

[Document(page_content='Now in Preview: Groq’s First Compound AI System. \n\n\n\n\n\n\nTry Now &\n\n\n\n\n\n\n\n\n\n\nLearn More. \n\n\n\n\n\n\n\n\n\n\n\n\n\nLlama 4 is live on GroqCloud™. \n\n\n\n\n\n\nLearn more &\n\n\n\n\n\n\n\n\n\n\ntry now.\n\n\n\n\n\n\n\n\n\n\n\n\n\nGroq Launches First TTS Model, PlayAI Dialog, Making Voice AI Sound More Human.  \n\n\n\n\n\n\nTry Now &\n\n\n\n\n\n\n\n\n\n\nLearn More\n\n\n\n\n\n\n\n\n\n\n\n\n\nGroqCloud™ Self-serve Developer Tier now available! \n\n\n\n\n\n\nLearn more.\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n\n\n\n \n\n \n\n\n\n\n\n \n\t\t\t\t\t\t\tProducts\t\t\t\t\t\t\n \n\n Close Products\n\n\n Open Products\n\n\n\n\n\n\n\n\n\n\nPricing\n\n\n\n\nProducts Overview\n\n\n\n\nEnterprise Access\n\n\n\n\nGroqCloud™ Platform\n\n\n\n\nGroqRack™ Cluster\n\n\n\n\n\n\n\n\n\n\n \n\t\t\t\t\t\t\tDevelopers\t\t\t\t\t\t\n \n\n Close Developers\n\n\n Open Developers\n\n\n\n\n\n\n\n\n\n\n\nFree API Key \xa0\n\n\n\n\nStart Building \xa0\n\n\n\n\nDiscord  \xa0\n\n\n\n\n

In [25]:
vectorStore =  Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="check_conversation")
retriever = vectorStore.as_retriever

In [26]:
## Prompt Template 
system_prompt = """You are a helpful assistant. You will be given a question and a context. Your task is to answer the question based on the context provided.
If the context does not contain enough information to answer the question, respond with "I don't know".
"{context}"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [33]:
from langchain.chains import create_retrieval_chain
retriever = vectorStore.as_retriever
combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever(), combine_docs_chain=combine_docs_chain)


In [40]:
response = rag_chain.invoke({"input": "whats is they are assembling?"})

In [41]:
response

{'input': 'whats is they are assembling?',
 'context': [Document(page_content='Videos\n\n\n\n\n\n\n\n\n\n\n\n \n\t\t\t\t\t\t\tAbout\t\t\t\t\t\t\n \n\n Close About\n\n\n Open About\n\n\n\n\n\n\n\n\n\n\nAbout Groq\n\n\n\n\nIn the News\n\n\n\n\nTeam\n\n\n\n\nCareers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDev\xa0Console\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\nGroq Hires Exceptional Talent \nAt Groq, we believe AI will change humanity forever, and that making it affordable and universally accessible is the key to human agency in an AI economy.\xa0We’re assembling a team of world-class engineers and business minds who believe in this mission. We’re hiring problem-solvers with the skills and desire to build a business that leaves a fingerprint on civilization.\xa0 \nOur Values \nWe value curiosity and humility. Every Groqster is a lifelong learner, eager to expand their capabilities while courageous enough to bring others along for the journey. If this resonates, explore opportunitie